In [1]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplcursors

# Getting Image Pixel coordinates using Mouse

In [27]:
import cv2
from matplotlib import pyplot as plt
import matplotlib 
%matplotlib qt

# Acquire default dots per inch value of matplotlib
dpi = matplotlib.rcParams['figure.dpi']

img = cv2.imread(r'C:\Users\Piyush\Desktop\IITK_raw_image.png', 1)

# Determine the figures size in inches to fit your image
height, width, depth = img.shape
figsize = width / float(dpi), height / float(dpi)

plt.figure(figsize=figsize)
plot = plt.imshow(img)
plt.show()

In [3]:
points = []

cursor = mplcursors.cursor(plot, hover=False)
@cursor.connect("add")
def cursor_clicked(sel):
    # sel.annotation.set_visible(False)
    sel.annotation.set_text(
        f'Clicked on\nx: {sel.target[0]:.2f} y: {sel.target[1]:.2f}\nindex: {sel.target.index}')
    points.append(sel.target.index)
    print("Current list of points:", points)

plt.show()
print("Selected points:", points)

Selected points: []
Current list of points: [(3794, 6596)]
Current list of points: [(3794, 6596), (3739, 6525)]
Current list of points: [(3794, 6596), (3739, 6525), (3699, 6522)]
Current list of points: [(3794, 6596), (3739, 6525), (3699, 6522), (3713, 6724)]


In [4]:
#points = [(3795, 6597), (3738, 6524), (3699, 6522), (3712, 6724)]
points

[(3794, 6596), (3739, 6525), (3699, 6522), (3713, 6724)]

In [5]:
pixel2d = np.array(points,dtype="double")
pixel2d

array([[3794., 6596.],
       [3739., 6525.],
       [3699., 6522.],
       [3713., 6724.]])

# Known Parameters - Camera Matrix, World GCP coordinates, Image GCP Coordinates, Distortion Coefficients

In [6]:
# Put manually the known real world gcp points in the same order.
gcp_world = np.array([(423650.2, 2932475.9,0),(423608.7, 2932508.4, 0.0),(423608.1, 2932531.1, 0.0),(423724.0, 2932523.1, 0.0)])
#gcp_world = np.array([(0.0, 0.0, 0.0),(1.775, 0.0, 0.0),(1.775, 4.620, 0.0),(0.0, 4.620, 0.0)])
print("World Coordinated of GCps")
print(gcp_world)

# Image coordinates of GCPs - using opencv python
gcp_img = pixel2d
#gcp_img = np.array([(95, 264),(100, 539),(349, 259),(352, 535)],dtype="double")
gcp_img = np.array([(275, 204),(331, 204),(331, 308),(275, 308)],dtype="double")
print("Image coordinate of GCPs in pixel\image coordinate")
print(gcp_img)

#Camera Matrix   - from zhangs method
K = np.array([[517.0036881709533, 0, 320],[0, 517.0036881709533, 212],[0, 0, 1]])
print("Camera Matrix")
print(K)

# distortion coefficients
diss_coef = np.array([[0.1128663679798094],[-1.487790079922432],[0],[0],[2.300571896761067]])
#diss_coef = np.zeros((5,1))
print("Distortion Coefficients")
print(diss_coef)

World Coordinated of GCps
[[ 423650.2 2932475.9       0. ]
 [ 423608.7 2932508.4       0. ]
 [ 423608.1 2932531.1       0. ]
 [ 423724.  2932523.1       0. ]]
Image coordinate of GCPs in pixel\image coordinate
[[275. 204.]
 [331. 204.]
 [331. 308.]
 [275. 308.]]
Camera Matrix
[[517.00368817   0.         320.        ]
 [  0.         517.00368817 212.        ]
 [  0.           0.           1.        ]]
Distortion Coefficients
[[ 0.11286637]
 [-1.48779008]
 [ 0.        ]
 [ 0.        ]
 [ 2.3005719 ]]


# Finding Homography

In [7]:
def findHomography(world3d,image2d,K,dist):
    (success, rotation_vector, translation_vector) = cv2.solvePnP(world3d,image2d, K, dist, flags=cv2.SOLVEPNP_ITERATIVE)
    print("rotation_vector")
    print(rotation_vector)
    
    print("translation_vector")
    translation_vector
    print(translation_vector)
    
    rotation_matrix, jacobian = cv2.Rodrigues(rotation_vector)
    print("Rotation Matrix")
    print(rotation_matrix)
    
    extrinsic_matrix = cv2.hconcat((rotation_matrix, translation_vector))
    print("Extrinisic parameter matrix")
    print(extrinsic_matrix)
    
    projection_matrix = np.dot(K,extrinsic_matrix)
    print("Projection Matrix")
    print(projection_matrix)
    
    homography_matrix = np.delete(projection_matrix,2,axis = 1)
    return homography_matrix

In [8]:
homography_matrix = findHomography(gcp_world,gcp_img,K,diss_coef)
print("homography_matrix")
homography_matrix

rotation_vector
[[ 0.13833433]
 [-1.75740868]
 [-0.50497142]]
translation_vector
[[ -405605.36625477]
 [-2480012.16504066]
 [-1569100.78720331]]
Rotation Matrix
[[-0.25275807  0.17482189 -0.95160426]
 [-0.35700203  0.89728619  0.2596672 ]
 [ 0.89925687  0.40535763 -0.16438456]]
Extrinisic parameter matrix
[[-2.52758073e-01  1.74821888e-01 -9.51604258e-01 -4.05605366e+05]
 [-3.57002026e-01  8.97286186e-01  2.59667196e-01 -2.48001217e+06]
 [ 8.99256865e-01  4.05357629e-01 -1.64384560e-01 -1.56910079e+06]]
Projection Matrix
[[ 1.57085341e+02  2.20098002e+02 -5.44585970e+02 -7.11811722e+08]
 [ 6.07109104e+00  5.49836085e+02  9.93993714e+01 -1.61482480e+09]
 [ 8.99256865e-01  4.05357629e-01 -1.64384560e-01 -1.56910079e+06]]
homography_matrix


array([[ 1.57085341e+02,  2.20098002e+02, -7.11811722e+08],
       [ 6.07109104e+00,  5.49836085e+02, -1.61482480e+09],
       [ 8.99256865e-01,  4.05357629e-01, -1.56910079e+06]])

# Finding 3d coordinate of a given 2d image coordinate

In [9]:
def get3d(homography_matrix,image2d):
    inverse_homography_matrix = np.linalg.inv(homography_matrix)
    world3d = np.dot(inverse_homography_matrix,image2d)
    Z = world3d[2,0]
    world3d = world3d/Z
    return world3d

In [10]:
import cv2
from matplotlib import pyplot as plt
import matplotlib 
%matplotlib qt

# Acquire default dots per inch value of matplotlib
dpi = matplotlib.rcParams['figure.dpi']

img = cv2.imread(r'C:\Users\Piyush\Desktop\IITK_raw_image.png', 1)

# Determine the figures size in inches to fit your image
height, width, depth = img.shape
figsize = width / float(dpi), height / float(dpi)

plt.figure(figsize=figsize)
plot = plt.imshow(img)
plt.show()

In [28]:
test_points = []

cursor = mplcursors.cursor(plot, hover=False)
@cursor.connect("add")
def cursor_clicked(sel):
    # sel.annotation.set_visible(False)
    sel.annotation.set_text(
        f'Clicked on\nx: {sel.target[0]:.2f} y: {sel.target[1]:.2f}\nindex: {sel.target.index}')
    test_points.append(sel.target.index)
    print("Current list of points:", test_points)

plt.show()
print("Selected points:", test_points)

Selected points: []
Current list of points: [(3820, 6528)]
Current list of points: [(3820, 6528), (3713, 6724)]
Current list of points: [(3820, 6528), (3713, 6724), (4216, 6352)]
Current list of points: [(3820, 6528), (3713, 6724), (4216, 6352), (4168, 6447)]
Current list of points: [(3820, 6528), (3713, 6724), (4216, 6352), (4168, 6447), (4148, 6249)]


In [29]:
 test_points

[(3820, 6528), (3713, 6724), (4216, 6352), (4168, 6447), (4148, 6249)]

In [30]:
# we took pixel coordinates of 3 points whose original coordinates are - (90,30),(90,20),(90,10)
pixel_test2d = test_points
tot_pts = len(pixel_test2d)
print(tot_pts)
pixel_test2d

5


[(3820, 6528), (3713, 6724), (4216, 6352), (4168, 6447), (4148, 6249)]

In [31]:
pixel_test2d = np.array(pixel_test2d,dtype="double").reshape(tot_pts,2)
pixel_test2d

array([[3820., 6528.],
       [3713., 6724.],
       [4216., 6352.],
       [4168., 6447.],
       [4148., 6249.]])

In [32]:
# test images
test_img = pixel_test2d

In [33]:
# converting to homogenous coordinates
a = test_img
b = np.ones((tot_pts,3))
b[:,:-1] = a
b

array([[3.820e+03, 6.528e+03, 1.000e+00],
       [3.713e+03, 6.724e+03, 1.000e+00],
       [4.216e+03, 6.352e+03, 1.000e+00],
       [4.168e+03, 6.447e+03, 1.000e+00],
       [4.148e+03, 6.249e+03, 1.000e+00]])

In [34]:
tot3d = []
for i in range(tot_pts):
    image2d = b[i]
    image2d = image2d.reshape(-1,1)
    tot3d.append(get3d(homography_matrix,image2d))

In [35]:
#tot3d  # 3d coordinates calculated from homography

In [36]:
np.array(tot3d).reshape(tot_pts,3)   # original coordinates are - [[90,30],[90,20],[90,10]]

array([[4.23035733e+05, 2.93247778e+06, 1.00000000e+00],
       [4.23024619e+05, 2.93250690e+06, 1.00000000e+00],
       [4.23053744e+05, 2.93242870e+06, 1.00000000e+00],
       [4.23050373e+05, 2.93243745e+06, 1.00000000e+00],
       [4.23053962e+05, 2.93242889e+06, 1.00000000e+00]])

In [ ]:
# Calculated - (423035.7,2932477.7),(423024.6,2932506.9),(423053.7,2932428.7),(423050.3,2932437.4),(4230539.6,2932428.8)
# actual -     (423610.4,2932460.6),(423724,2932523.1),(423509.1,2932231.2),(423562.4,2932259.2),(423449.1,2932270.6)

# Using Inbuilt findHomograaphy() method

In [39]:
h, status = cv2.findHomography(gcp_world,gcp_img )
h

array([[-1.64242881e-04, -8.17483636e-05,  3.09297996e+02],
       [-1.97470024e-04, -1.39893860e-04,  4.93887186e+02],
       [-6.27071735e-07, -2.50427414e-07,  1.00000000e+00]])

In [40]:
homography_matrix

array([[ 1.57085341e+02,  2.20098002e+02, -7.11811722e+08],
       [ 6.07109104e+00,  5.49836085e+02, -1.61482480e+09],
       [ 8.99256865e-01,  4.05357629e-01, -1.56910079e+06]])

In [43]:
N = tot_pts
tot3d = []
for i in range(N):
    image2d = b[i]
    image2d = image2d.reshape(-1,1)
    tot3d.append(get3d(h,image2d))

In [44]:
np.array(tot3d).reshape(tot_pts,3)    # original coordinates are - [[90,30],[90,20],[90,10]]

array([[4.23550000e+05, 2.93260412e+06, 1.00000000e+00],
       [4.23546558e+05, 2.93261299e+06, 1.00000000e+00],
       [4.23555769e+05, 2.93258915e+06, 1.00000000e+00],
       [4.23554718e+05, 2.93259185e+06, 1.00000000e+00],
       [4.23555780e+05, 2.93258916e+06, 1.00000000e+00]])

In [ ]:
# calculated - (423550.0,2932604.1),(423546.5,2932612.9),(423555.7,2932589.1),(423554.7,2932591.8),(423555.7,2932589.1)
# actual -     (423610.4,2932460.6),(423724,2932523.1),(423509.1,2932231.2),(423562.4,2932259.2),(423449.1,2932270.6)